# Lecture5; Multilayer Perceptron

## Multilayer Perceptron

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

### Data processing

### Define Neural Network

In [ ]:
# Prepare MNIST data.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Convert to float32.
x_train, x_test = tf.cast(x_train, tf.float32), tf.cast(x_test, tf.float32)

num_classes = 10
num_features = x_train.shape[1] * x_train.shape[2]

print(num_classes)
print(num_features)

# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = tf.reshape(x_train,[-1, num_features]), tf.reshape(x_test, [-1, num_features])
# Normalize images value from [0, 255] to [0, 1].


x_train, x_test = x_train / 255., x_test / 255.

In [ ]:
# #만약에 tensorflow.keras.Model 혹은 tensorflow.keras.layers를 사용하지 않고 직접 Neural Network를 코딩해야한다면

# W1=tf.Variable(tf.random_normal_initializer(stddev=1)([num_features, 128])),
# W2=tf.Variable(tf.random_normal_initializer(stddev=1)([128, 256])),
# W3=tf.Variable(tf.random_normal_initializer(stddev=1)([256, num_classes]))
# b1=tf.Variable(tf.zeros([128])),
# b2=tf.Variable(tf.zeros([256])),
# b3=tf.Variable(tf.zeros([num_classes]))

# def neural_net(x):
#     layer_1 = tf.matmul(x, W1) + b1
#     layer_1 = tf.nn.relu(layer_1)
    
#     layer_2 = tf.matmul(layer_1, W2) + b2
#     layer_2 = tf.nn.relu(layer_2)
    
#     out_layer = tf.matmul(layer_2, W3) + b3
#     return tf.nn.softmax(out_layer)

In [ ]:
# tensorflow.keras.Model 혹은 tensorflow.keras.layers를 사용해서 간단하게
from tensorflow.keras import Model, layers

class NeuralNet(Model):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = layers.Dense(128, activation=tf.nn.relu)
        self.fc2 = layers.Dense(256, activation=tf.nn.relu)
        self.out = layers.Dense(num_classes)

    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        if not is_training:
            x = tf.nn.softmax(x) 
        return x

neural_net = NeuralNet()

In [ ]:
def cross_entropy_loss(x, y): # with_logits --> logits를 쓰는게 numerical stable하다
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64)) 
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
lr = 0.003
optimizer = tf.optimizers.SGD(lr)

def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = neural_net(x, is_training=True)
        loss = cross_entropy_loss(pred, y)
        
    trainable_variables = neural_net.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
# Use tf.data API to shuffle and batch data.
batch_size = 200
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size).prefetch(1)

epoch = 40
display_epoch = 4

# Run training for the given number of steps.
for epo in range(1,epoch+1):
    for step, (batch_x, batch_y) in enumerate(train_data, 1):
        run_optimization(batch_x, batch_y)
        
    if epo % display_epoch == 0:
        pred = neural_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("epoch: ", epo, ", loss: ", loss.numpy(), ", acc: ", acc.numpy())

In [ ]:
# Test model on validation set.
pred = neural_net(x_test)
print("Test Accuracy: ", accuracy(pred, y_test).numpy())

In [ ]:
# Predict 5 images from validation set.
n_images = 5
test_images = x_test[:n_images]
predictions = neural_net(test_images)

# Display image and model prediction.
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction: %i" % np.argmax(predictions.numpy()[i]))

## model.fit방식으로 구현

In [ ]:
model2=tf.keras.Sequential(                           # build block
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(256, actication='relu'),
     tf.keras.layers.Dense(10)
     ])

In [ ]:
model2.compile(optimizer='SGD', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
               , metrics=['accuracy'])                                    # 어떤 loss func을 사용하고, metric, optimizer를 사용할지 결정

In [ ]:
inform=model2.fit(x_train, y_train, batch_size=128, epochs=40)            # batch size는 default가 32라고 함, val dataset을 넣어줄 수도 있고, 자동으로 쪼개서 쓰기도 함, validation_split=0.2


fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(inform.history['loss'], 'y', label='train loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')

acc_ax.plot(inform.history['accuracy'], 'b', label='train acc')


plt.show()


test_loss, test_acc = model2.evaluate(x_test, y_test) # verbose = 0 --> ignore, 1 --> progress bare, 2--> one line per epoch